<a href="https://colab.research.google.com/github/killian31/NLP_Project105/blob/main/Multilingual_machine_reading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 1




## 1.1 Preprocessing

In [ ]:
!pip install datasets
!pip install spacy
!python -m spacy download ja_core_news_sm 
!python -m spacy download fi_core_news_sm

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import re
import spacy 
import numpy as np
import pandas as pd
from datasets import load_dataset

In [ ]:
dataset = load_dataset("copenlu/answerable_tydiqa")
train_set = dataset["train"]
validation_set = dataset["validation"]

train_set_pd = train_set.to_pandas()
df = train_set_pd[train_set_pd.language.isin(['finnish','english','japanese'])]

validation_set_pd = validation_set.to_pandas()
df_val = validation_set_pd[validation_set_pd.language.isin(['finnish','english','japanese'])]

  0%|          | 0/2 [00:00<?, ?it/s]


### 1.1.a Tokenization of english, japanese and finnish.

In [ ]:
# Load relevant languages
nlp_en = spacy.load('en_core_web_sm')
nlp_ja = spacy.load('ja_core_news_sm')
nlp_fi = spacy.load('fi_core_news_sm')

In [ ]:
# Function to tokenize text
def tokenizer_lang(text, lang):
  if lang in ['english','japanese','finnish']:
    if lang == 'english':
      doc = nlp_en.tokenizer(text)
    elif lang == 'japanese':
      doc = nlp_ja.tokenizer(text)
    elif lang == 'finnish':
      doc = nlp_fi.tokenizer(text)

    return [t.text for t in doc if not t.is_punct]

In [ ]:
def tokenize_col(df, cols):
  for col in cols:
    df.loc[:, col+'_t'] = df.apply(lambda x: tokenizer_lang(x[col], x['language']), axis=1)

In [ ]:
tokenize_col(df, ['question_text', 'document_plaintext'])
tokenize_col(df_val, ['question_text', 'document_plaintext'])

In [ ]:
df.head()

,question_text,document_title,language,annotations,document_plaintext,document_url,question_text_t,document_plaintext_t
0,Milloin Charles Fort syntyi?,Charles Fort,finnish,"{'answer_start': [18], 'answer_text': ['6. elo...",Charles Hoy Fort (6. elokuuta (joidenkin lähte...,https://fi.wikipedia.org/wiki/Charles%20Fort,"[Milloin, Charles, Fort, syntyi]","[Charles, Hoy, Fort, 6, elokuuta, joidenkin, l..."
1,“ダン” ダニエル・ジャドソン・キャラハンの出身はどこ,ダニエル・J・キャラハン,japanese,"{'answer_start': [35], 'answer_text': ['カリフォルニ...",“ダン”こと、ダニエル・ジャドソン・キャラハンは1890年7月26日、カリフォルニア州サンフ...,https://ja.wikipedia.org/wiki/%E3%83%80%E3%83%...,"[ダン, ダニエル, ジャドソン, キャラハン, の, 出身, は, どこ]","[ダン, こと, ダニエル, ジャドソン, キャラハン, は, 1890, 年, 7, 月,..."
10,Mitä on altruismi?,Altruismi,finnish,"{'answer_start': [44], 'answer_text': ['epäits...","\n\n\nAltruismi ([1], ”toinen”[2]) tarkoittaa...",https://fi.wikipedia.org/wiki/Altruismi,"[Mitä, on, altruismi]","[\n\n\n, Altruismi, 1, , toinen”[2, tarkoitta..."
12,Mikä oli Wilhelm Wagner viimeinen sävellys?,Richard Wagner,finnish,"{'answer_start': [597], 'answer_text': ['Parsi...",Wagnerin mestariteoksia ovat hänen myöhäiskaud...,https://fi.wikipedia.org/wiki/Richard%20Wagner,"[Mikä, oli, Wilhelm, Wagner, viimeinen, sävellys]","[Wagnerin, mestariteoksia, ovat, hänen, myöhäi..."
13,Missä Harz sijaitsee?,Harz,finnish,"{'answer_start': [25], 'answer_text': ['Pohjoi...",\n\nHarz on horstivuoristo Pohjois-Saksassa[1]...,https://fi.wikipedia.org/wiki/Harz,"[Missä, Harz, sijaitsee]","[\n\n, Harz, on, horstivuoristo, Pohjois-Saksa..."


In [ ]:
df_val.head()

,question_text,document_title,language,annotations,document_plaintext,document_url,question_text_t,document_plaintext_t,answerable,answerable_t
3,化学兵器禁止条約はどこで採択された？,化学兵器禁止条約,japanese,"{'answer_start': [11], 'answer_text': ['パリ']}",1993年1月13日にパリにおいて署名がなされ、1997年4月29日に発効した[1]。実効的...,https://ja.wikipedia.org/wiki/%E5%8C%96%E5%AD%...,"[化学, 兵器, 禁止, 条約, は, どこ, で, 採択, さ, れ, た]","[1993, 年, 1, 月, 13, 日, に, パリ, に, おい, て, 署名, が,...",1,Answerable
9,オリヴィア・デ・ハヴィランドが生まれたのはいつ,オリヴィア・デ・ハヴィランド,japanese,"{'answer_start': [46], 'answer_text': ['1916年7...","\nオリヴィア・デ・ハヴィランド（Dame Olivia De Havilland, DBE...",https://ja.wikipedia.org/wiki/%E3%82%AA%E3%83%...,"[オリヴィア, デ, ハヴィランド, が, 生まれ, た, の, は, いつ]","[\n, オリヴィア, デ, ハヴィランド, Dame, Olivia, De Havill...",1,Answerable
11,Kauanko lasia on valmistettu?,Lasi,finnish,"{'answer_start': [160], 'answer_text': ['noin ...",Vanhin tunnettu lasilaatu on alkali­kalkki­las...,https://fi.wikipedia.org/wiki/Lasi,"[Kauanko, lasia, on, valmistettu]","[Vanhin, tunnettu, lasilaatu, on, alkali­kalkk...",1,Answerable
14,Mikä on Ponzi-huijaus?,Ponzi-huijaus,finnish,"{'answer_start': [39], 'answer_text': ['pyrami...",Ponzi-huijaus eli Ponzi-järjestelmä on pyramid...,https://fi.wikipedia.org/wiki/Ponzi-huijaus,"[Mikä, on, Ponzi-huijaus]","[Ponzi-huijaus, eli, Ponzi-järjestelmä, on, py...",1,Answerable
20,Mikä oli Napoleonin sotien lopputulos?,Napoleonin sodat,finnish,"{'answer_start': [15], 'answer_text': ['Ranska...",Sotien jälkeen Ranska alistettiin kovilla rauh...,https://fi.wikipedia.org/wiki/Napoleonin%20sodat,"[Mikä, oli, Napoleonin, sotien, lopputulos]","[Sotien, jälkeen, Ranska, alistettiin, kovilla...",1,Answerable


**Remember to preprocess the document_plaintext tokens list (removing brackets, ...)**

### 1.1.b Identify most common first and last words

In [ ]:
df.loc[:,'first_token'] = df['tokens'].apply(lambda x: x[0])
df.loc[:,'last_token'] = df['tokens'].apply(lambda x: x[-1])

In [ ]:
print('Most common first word:')
print(df.groupby(['language'])['first_token'].agg(pd.Series.mode))

print('\nFive most common first words:')
print(df.groupby(['language'])['first_token'].head(5))

In [ ]:
print('Most common last word:')
print(df.groupby(['language'])['last_token'].agg(pd.Series.mode))

print('\nFive most common last words:')
print(df.groupby(['language'])['last_token'].head(5))

## 1.2 Binary classification

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(123)

In [ ]:
!pip3 install transformers

In [ ]:
!pip install fugashi[unidic-lite]

In [ ]:
from transformers import AutoTokenizer
mbert_tokeniser = AutoTokenizer.from_pretrained("bert-base-multilingual-uncased")

In [ ]:
label_to_ix = {'Non-answerable':0, 'Answerable':1}

In [ ]:
df['answerable'] = df.annotations.apply(lambda x: 0 if x['answer_start']==-1 else 1)
df_val['answerable'] = df_val.annotations.apply(lambda x: 0 if x['answer_start']==-1 else 1)

df['answerable_t'] = df.annotations.apply(lambda x: 'Non-answerable' if x['answer_start']==-1 else 'Answerable')
df_val['answerable_t'] = df_val.annotations.apply(lambda x: 'Non-answerable' if x['answer_start']==-1 else 'Answerable')

### 1.2.a Design classifier

In [ ]:
tokenizer_fi = AutoTokenizer.from_pretrained("TurkuNLP/bert-base-finnish-cased-v1")
#tokenizer_ja = AutoTokenizer.from_pretrained("cl-tohoku/bert-base-japanese") # Issue
tokenizer_en = AutoTokenizer.from_pretrained("Jean-Baptiste/roberta-large-ner-english")

In [ ]:
class BoWClassifier(nn.Module):

    def __init__(self, num_labels, vocab_size, num_hidden = 2):
        # Calls the init function of nn.Module. 
        super(BoWClassifier, self).__init__()

        # Define the parameters that you need.
        self.linear = nn.Linear(vocab_size, num_hidden)
        # non-linearity (here it is also a layer!)
        self.nonlinear = nn.ReLU()
        # final affine transformation
        self.final = nn.Linear(num_hidden, num_labels)

    def forward(self, bow_vec):
        # Pass the input through the linear layer,
        # then pass that through log_softmax.
        # Many non-linearities and other functions are in torch.nn.functional
        return self.final(self.nonlinear(self.linear(bow_vec)))

In [ ]:
# Functions to create BoW vectors
def make_bow_vector(sentence, word_to_ix):
    vec = torch.zeros(len(word_to_ix))
    for word in sentence:
        vec[word_to_ix[word]] += 1
    return vec.view(1, -1)


def make_target(label, label_to_ix):
    return torch.LongTensor([label_to_ix[label]])

Test model is running ...

In [ ]:
# Function to map each word in the vocab to an unique integer
# Indexing the Bag of words vector
word_to_ix = {}
for sent in df['question_text_t'] + df['document_plaintext_t']:# + df_val['question_text_t'] + df_val['document_plaintext_t']:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
print(word_to_ix)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
VOCAB_SIZE = len(word_to_ix)
NUM_LABELS = 2

In [ ]:
model = BoWClassifier(NUM_LABELS, VOCAB_SIZE)

# The first output below is A, the second is b.
for param in model.parameters():
    print(param)

Parameter containing:
tensor([[-7.6024e-04,  6.1756e-05, -9.2594e-04,  ...,  1.6241e-03,
         -1.5288e-03,  7.7469e-04],
        [ 1.0381e-03, -1.5680e-03, -1.6714e-03,  ...,  1.2232e-03,
         -1.7472e-03, -3.9226e-04]], requires_grad=True)
Parameter containing:
tensor([ 0.0016, -0.0011], requires_grad=True)
Parameter containing:
tensor([[-0.1705,  0.3914],
        [-0.6458, -0.5898]], requires_grad=True)
Parameter containing:
tensor([-0.0808, -0.0233], requires_grad=True)


In [ ]:
test_data = df.loc[df.language == 'finnish',['question_text_t','answerable']].head(10).values.tolist()

In [ ]:
with torch.no_grad():
    for text, label in test_data:
        bow_vec = make_bow_vector(text, word_to_ix)
        log_probs = model(bow_vec)
        print(log_probs)

# Print the matrix column corresponding to "Charles"
print(next(model.parameters())[:, word_to_ix["Charles"]])

tensor([[-0.0733, -0.4155]])
tensor([[-0.0720, -0.4135]])
tensor([[-0.0721, -0.4137]])
tensor([[-0.0715, -0.4141]])
tensor([[-0.0720, -0.4145]])
tensor([[-0.0724, -0.4142]])
tensor([[-0.0725, -0.4143]])
tensor([[-0.0724, -0.4142]])
tensor([[-0.0720, -0.4135]])
tensor([[-0.0727, -0.4146]])
tensor([0.0012, 0.0010], grad_fn=<SelectBackward0>)


### 1.2.b Train classifier

In [ ]:
# Model for each language
word_to_ix_fi = tokenizer_fi.vocab
model_fi = BoWClassifier(NUM_LABELS, len(word_to_ix_fi))

word_to_ix_en = tokenizer_en.vocab
model_en = BoWClassifier(NUM_LABELS, len(word_to_ix_en))

#word_to_ix_ja = tokenizer_ja.vocab
#model_ja = BoWClassifier(NUM_LABELS, len(word_to_ix_ja))

In [ ]:
df_lang = df.loc[df.language == 'finnish',['question_text_t','answerable_t']].values.tolist()

In [ ]:
for epoch in range(epochs):
    for instance, label in df_lang:
        # Step 1. Remember that PyTorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Make our BOW vector and also we must wrap the target in a
        # Tensor as an integer. For example, if the target is SPANISH, then
        # we wrap the integer 0. The loss function then knows that the 0th
        # element of the log probabilities is the log probability
        # corresponding to SPANISH
        bow_vec = make_bow_vector(instance, word_to_ix)
        target = make_target(label, label_to_ix)

        # Step 3. Run our forward pass.
        log_probs = model(bow_vec)

        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss = loss_function(log_probs, target)
        loss.backward()
        optimizer.step()

In [ ]:
def train_loop(df, model, loss_fn, optimizer, word_to_ix, label_to_ix):
  for X, y in df:
    # Step 1: Clear gradients before instance
    model.zero_grad()

    # Step 2: Make BOW vector and wrap in a Tensor as an integer
    bow_vec = make_bow_vector(X, word_to_ix)
    target = make_target(y, label_to_ix)

    # Step 3: Run forward pass
    pred = model(bow_vec)

    # Step 4: Compute loss, gradients, and update the parameters
    loss = loss_fn(pred, target)
    loss.backward()
    optimizer.step()

In [ ]:
def test_loop(df, model, loss_fn, word_to_ix, label_to_ix):
  size = len(df)
  #num_batches = len(???)
  test_loss, correct = 0, 0

  with torch.no_grad():
    for X, y in df:
      bow_vec = make_bow_vector(X, word_to_ix)
      target = make_target(y, label_to_ix)

      pred = model(bow_vec)
      test_loss += loss_fn(pred, target).item()
      correct += (pred.argmax(1) == target).type(torch.float).sum().item()
  
  #test_loss /= num_batches
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%")

In [ ]:
VOCAB_SIZE = len(word_to_ix)
NUM_LABELS = 2

model = BoWClassifier(NUM_LABELS, VOCAB_SIZE)

In [ ]:
epochs = 5
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [ ]:
for t in range(epochs):
  print(f"Epoch {t+1}\n--------------")
  train_loop(df_lang, model, loss_function, optimizer, word_to_ix, label_to_ix)
  #test_loop(df_lang, model, loss_function, word_to_ix, label_to_ix)
print('Done!')

Epoch 1
--------------
Epoch 2
--------------
Epoch 3
--------------
Epoch 4
--------------
Epoch 5
--------------
Done!


Model is not improving...

### 1.2.c Evaluate classifier

In [ ]:
test_loop(df_lang, model, loss_function, word_to_ix, label_to_ix)

Test Error: 
 Accuracy: 50.0%


In [ ]:
with torch.no_grad():
    for instance, label in test_data:
        bow_vec = make_bow_vector(instance, word_to_ix)
        log_probs = model(bow_vec)
        print(log_probs)

tensor([[ 3.5541, -3.6579]])
tensor([[ 3.5541, -3.6579]])
tensor([[ 3.5541, -3.6579]])
tensor([[ 3.5541, -3.6579]])
tensor([[ 3.5541, -3.6579]])
tensor([[ 3.5541, -3.6579]])
tensor([[ 3.5541, -3.6579]])
tensor([[ 3.5541, -3.6579]])
tensor([[ 3.5541, -3.6579]])
tensor([[ 3.5541, -3.6579]])


Calculate accuracy

# Assignment 2 
Representation Learning

In [ ]:
from tqdm import tqdm
import nltk
from nltk.corpus import stopwords

In [ ]:
def get_bpemb_features(dataset, bpemb):
  # With bpemb we can tokenize and embed an entire document using .embed(x)
  X = [bpemb.embed(x).mean(0) for x in tqdm(dataset[:,0])]
  y = list(dataset[:,1])
  return X,y

In [ ]:
def get_glove_features(dataset, glove, nlp):
  X = []
  for x in tqdm(dataset[:,0]):
    # For glove embeddings, we first tokenize the sentence using spacy, lower-case and remove stopwords, and get the available word vectors
    vecs = np.vstack([glove[t.text.lower()] if t.text.lower() in glove else np.zeros(100) for t in nlp(x) if t.text.lower() not in stopwords])
    X.append(vecs.mean(0))
  y = list(dataset[:,1])
  return np.stack(X),y